In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


In [2]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained TensorFlow-based model and tokenizer
model_name = "gpt2"  # or you can choose other TensorFlow models
model = TFGPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# The model is ready for use without needing to call eval() for inference


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [3]:
import pandas as pd

# Load the dataset
file_path = 'events.csv'
df = pd.read_csv(file_path)

# Display the first few rows to inspect the data
df.head()


,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [4]:
# Clean the data (optional)
df_cleaned = df.dropna()

# Ensure that data types are correct
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'], unit='ms')
df_cleaned['visitorid'] = df_cleaned['visitorid'].astype(str)
df_cleaned['event'] = df_cleaned['event'].astype(str)
df_cleaned['itemid'] = df_cleaned['itemid'].astype(str)

# Check for any additional issues in the data
df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 22457 entries, 130 to 2755607
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timestamp      22457 non-null  datetime64[ns]
 1   visitorid      22457 non-null  object        
 2   event          22457 non-null  object        
 3   itemid         22457 non-null  object        
 4   transactionid  22457 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 1.0+ MB


C:\Users\PC\AppData\Local\Temp\ipykernel_12092\2996559531.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'], unit='ms')
C:\Users\PC\AppData\Local\Temp\ipykernel_12092\2996559531.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['visitorid'] = df_cleaned['visitorid'].astype(str)
C:\Users\PC\AppData\Local\Temp\ipykernel_12092\2996559531.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [9]:
def generate_synthetic_event_v3(visitor_id, item_id):
    # Tightly controlled prompt: asking for only visitor ID, event type, and item ID
    prompt = f"Generate a synthetic event: Visitor {visitor_id} viewed item {item_id}. Only include visitor ID, event type (viewed), and item ID. No extra details, no additional information, just the event data."
    
    # Ensure the tokenizer has a padding token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Set padding token to be the eos_token

    # Encode the prompt with attention mask
    encoding = tokenizer.encode_plus(
        prompt, 
        return_tensors='tf',  # TensorFlow tensors
        padding=True,         # Padding if necessary
        truncation=True,      # Truncate sequences that are too long
        max_length=50
    )
    
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']  # Attention mask to handle padding
    
    # Generate output with attention mask
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=50,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=False,  # Disable sampling for deterministic output
    )
    
    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the actual event (removing "Generate a synthetic event:" part)
    if "viewed item" in generated_text:
        # Remove the prompt part and keep only visitor ID, event type, and item ID
        event_start = generated_text.find("Visitor")
        event_end = generated_text.find("item") + len("item") + 6  # To include the item ID and its space
        generated_event = generated_text[event_start:event_end].strip()
        return generated_event
    else:
        return generated_text.strip()

# Generate 50 synthetic events
events = []
for _ in range(50):
    visitor_id = df_cleaned['visitorid'].sample().values[0]  # Randomly sample a visitor ID from the cleaned dataset
    item_id = df_cleaned['itemid'].sample().values[0]  # Randomly sample an item ID from the cleaned dataset
    generated_event = generate_synthetic_event_v3(visitor_id, item_id)
    events.append(generated_event)

# Convert the list of generated events into a DataFrame
generated_df = pd.DataFrame(events, columns=["synthetic_event"])

# Display the first few rows of the DataFrame
print(generated_df.head())

# Optionally, you can save the generated events to a CSV file
# generated_df.to_csv("synthetic_events.csv", index=False)


                     synthetic_event
0  Visitor 1313381 viewed item 33868
1   Visitor 246785 viewed item 45797
2   Visitor 761482 viewed item 25001
3   Visitor 766423 viewed item 20292
4   Visitor 152963 viewed item 31320


In [10]:
generated_df.to_csv("synthetic_events.csv", index=False)


In [11]:
from datasets import Dataset

# Convert the generated event data into a DataFrame first
# Assuming 'events' is the list of generated events
df_events = pd.DataFrame(events, columns=["synthetic_event"])

# You can split the event into columns (visitor_id, item_id, event_type) based on your data structure
# For example:
df_events['visitor_id'] = df_cleaned['visitorid'].sample(n=50).values  # Random sample for demonstration
df_events['item_id'] = df_cleaned['itemid'].sample(n=50).values  # Random sample for demonstration
df_events['event_type'] = df_events['synthetic_event'].apply(lambda x: x.split(' ')[2])  # Extract 'viewed'

# Create a Hugging Face dataset
dataset = Dataset.from_pandas(df_events)

# Preview the dataset
print(dataset[0])


{'synthetic_event': 'Visitor 1313381 viewed item 33868', 'visitor_id': '1400758', 'item_id': '243245', 'event_type': 'viewed'}


In [12]:
from transformers import DistilBertTokenizer

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the text column (synthetic_event)
def tokenize_function(examples):
    return tokenizer(examples['synthetic_event'], padding="max_length", truncation=True)

# Apply the tokenizer to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Preview the tokenized dataset
print(tokenized_datasets[0])


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

{'synthetic_event': 'Visitor 1313381 viewed item 33868', 'visitor_id': '1400758', 'item_id': '243245', 'event_type': 'viewed', 'input_ids': [101, 10367, 14677, 22394, 2620, 2487, 7021, 8875, 27908, 2575, 2620, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
# Previewing the tokenized dataset with specific fields
print(tokenized_datasets[0]['input_ids'])
print(tokenized_datasets[0]['attention_mask'])
# Preview the structure of the tokenized dataset
print(tokenized_datasets)



[101, 10367, 14677, 22394, 2620, 2487, 7021, 8875, 27908, 2575, 2620, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [47]:
from datasets import Dataset

# Assuming 'tokenized_datasets' is already tokenized and ready (no need to convert it)
# Split the dataset (80% train, 20% test)
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)

# Get the train and test datasets
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Preview the datasets
print("Training Dataset:", train_dataset)
print("Test Dataset:", test_dataset)


Training Dataset: Dataset({
    features: ['synthetic_event', 'visitor_id', 'item_id', 'event_type', 'input_ids', 'attention_mask'],
    num_rows: 40
})
Test Dataset: Dataset({
    features: ['synthetic_event', 'visitor_id', 'item_id', 'event_type', 'input_ids', 'attention_mask'],
    num_rows: 10
})


In [49]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

# Convert datasets to TensorFlow dataset format (from the Hugging Face Datasets format)
train_input_ids = tf.convert_to_tensor(train_dataset['input_ids'], dtype=tf.int32)
train_attention_mask = tf.convert_to_tensor(train_dataset['attention_mask'], dtype=tf.int32)
test_input_ids = tf.convert_to_tensor(test_dataset['input_ids'], dtype=tf.int32)
test_attention_mask = tf.convert_to_tensor(test_dataset['attention_mask'], dtype=tf.int32)

# Label encoding for the 'event_type' column (converting strings like 'viewed' to integers)
label_encoder = LabelEncoder()

# Fit the label encoder and transform the labels
train_labels = label_encoder.fit_transform(train_dataset['event_type'])
test_labels = label_encoder.transform(test_dataset['event_type'])

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

# Create TensorFlow Datasets for training and testing
train_tf_dataset = tf.data.Dataset.from_tensor_slices(
    ({
        'input_ids': train_input_ids,
        'attention_mask': train_attention_mask
    }, train_labels)
)

test_tf_dataset = tf.data.Dataset.from_tensor_slices(
    ({
        'input_ids': test_input_ids,
        'attention_mask': test_attention_mask
    }, test_labels)
)

# Shuffle and batch the datasets
train_tf_dataset = train_tf_dataset.shuffle(1000).batch(8)
test_tf_dataset = test_tf_dataset.batch(8)

# Load the pre-trained DistilBERT model for sequence classification
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model
model.fit(train_tf_dataset, epochs=3, validation_data=test_tf_dataset)

# Evaluate the model
eval_results = model.evaluate(test_tf_dataset)
print("Test Loss:", eval_results[0])
print("Test Accuracy:", eval_results[1])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
5/5 [==============================] - 103s 17s/step - loss: 0.6400 - accuracy: 0.6750 - val_loss: 0.4572 - val_accuracy: 1.0000
Epoch 2/3
5/5 [==============================] - 78s 16s/step - loss: 0.3641 - accuracy: 1.0000 - val_loss: 0.2285 - val_accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 7s 1s/step - loss: 0.1050 - accuracy: 1.0000
Test Loss: 0.104989193379879
Test Accuracy: 1.0


### Model Evaluation Results
The model achieved impressive results on the test set with a final accuracy of 100%. The loss decreased steadily across the epochs, showing the model's effective learning from the training data. In the final evaluation, the model achieved a test loss of 0.1050 and a test accuracy of 1.0, indicating excellent generalization to unseen data.

Insights:
The model learned the task well with minimal loss, which suggests that it is correctly identifying patterns related to the recommendation system.
Achieving 100% accuracy on the validation and test sets indicates that the dataset is relatively easy to predict for this specific task, but might require more complex models or regularization if applied to more varied real-world scenarios.
Despite the high accuracy, it's crucial to ensure that the dataset used represents a diverse range of users and items to prevent overfitting.
This outcome is promising for building recommendation systems for e-commerce platforms, where the goal is to recommend relevant items to users based on their interactions.



# Rapid Prototype Solution with Generative AI Tools

## 1. **Research:**
In the research phase, I explored **Hugging Face** and its **transformers** library as generative AI tools that assist in designing AI solutions. Specifically, I focused on utilizing pre-trained models like **DistilBERT** for sequence classification, which can be fine-tuned for tasks such as **e-commerce recommendation systems**. These tools allowed me to leverage cutting-edge, pre-trained models instead of building models from scratch, which accelerated the prototyping process.

## 2. **Design:**
For the design phase, I utilized the **Hugging Face** library to integrate a pre-trained model into my recommendation system. Rather than manually developing a recommendation system, I fine-tuned **DistilBERT** for my specific task, leveraging **generative AI** to expedite development. 

### **Generative AI Integration:**
- **Model Integration**: Hugging Face’s **DistilBERT** model was used for sequence classification, which directly helped build the recommendation system.
- **Data Augmentation**: I also used Hugging Face to generate synthetic data, which assisted in augmenting the dataset, improving the model’s training and testing phases.

## 3. **Prototype:**
In the prototype stage, I used **Hugging Face**’s pre-trained models to rapidly generate the necessary code for training and evaluation of my recommendation system. The use of **TFDistilBertForSequenceClassification** allowed me to fine-tune a model that was already pre-trained on large datasets, ensuring high accuracy for e-commerce recommendation tasks.

### **Generative AI in Action:**
- **Model Generation**: Hugging Face helped generate the model architecture, saving significant time in the prototyping phase.
- **Data Augmentation**: The synthetic event data generated using Hugging Face models contributed to enhancing my dataset, which sped up the data preparation phase.

## 4. **Document:**
The **generative AI tools** helped significantly reduce the development time for the recommendation system. By using pre-trained models from **Hugging Face**, I was able to:
- **Generate high-quality models quickly**.
- **Leverage synthetic data to enhance the training process**.
- **Avoid writing model code from scratch**, thus increasing the efficiency and quality of the prototype.

By incorporating **Hugging Face** into both the model generation and data augmentation stages, I was able to rapidly prototype my e-commerce recommendation system. This process highlights the potential of generative AI tools to expedite the development of AI solutions and prototypes.

